# Mod 5 Online Shoppers Intent Project 

# Contents:
1. 
2.
3.
4.
5.
6.
7.


## 1. Business Case: 

## 2. Import Data

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from textwrap import wrap
%matplotlib inline


from sklearn.preprocessing import OneHotEncoder

from sklearn.model_selection import KFold, train_test_split
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
              
from sklearn.tree import DecisionTreeClassifier 
from sklearn.tree import export_graphviz

from IPython.display import Image  
from pydotplus import graph_from_dot_data

from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, precision_recall_curve



from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier


In [2]:
df = pd.read_csv('online_shoppers_intention.csv')
df.shape

(12330, 18)

## 3. Data Scrubbing

In [3]:
df.isnull().sum()

Administrative             0
Administrative_Duration    0
Informational              0
Informational_Duration     0
ProductRelated             0
ProductRelated_Duration    0
BounceRates                0
ExitRates                  0
PageValues                 0
SpecialDay                 0
Month                      0
OperatingSystems           0
Browser                    0
Region                     0
TrafficType                0
VisitorType                0
Weekend                    0
Revenue                    0
dtype: int64

In [4]:
# Check for Duplicated values
df.duplicated().sum()
# 125 values duplicated
# Will leave them in as they could be duplicates by chance and 
# they are very small portion of our data


125

# 4. Exploration

## 4.1 Data Set Information:

The dataset consists of feature vectors belonging to 12,330 sessions.
The dataset was formed so that each session
would belong to a different user in a 1-year period to avoid
any tendency to a specific campaign, special day, user
profile, or period.


### 4.2 Attribute Information:

The dataset consists of 10 numerical and 8 categorical attributes.
The 'Revenue' attribute can be used as the class label.

"Administrative", "Administrative Duration", "Informational", "Informational Duration", "Product Related" and "Product Related Duration" represent the number of different types of pages visited by the visitor in that session and total time spent in each of these page categories. The values of these features are derived from the URL information of the pages visited by the user and updated in real time when a user takes an action, e.g. moving from one page to another. The "Bounce Rate", "Exit Rate" and "Page Value" features represent the metrics measured by "Google Analytics" for each page in the e-commerce site. The value of "Bounce Rate" feature for a web page refers to the percentage of visitors who enter the site from that page and then leave ("bounce") without triggering any other requests to the analytics server during that session. The value of "Exit Rate" feature for a specific web page is calculated as for all pageviews to the page, the percentage that were the last in the session. The "Page Value" feature represents the average value for a web page that a user visited before completing an e-commerce transaction. The "Special Day" feature indicates the closeness of the site visiting time to a specific special day (e.g. Mother’s Day, Valentine's Day) in which the sessions are more likely to be finalized with transaction. The value of this attribute is determined by considering the dynamics of e-commerce such as the duration between the order date and delivery date. For example, for Valentina’s day, this value takes a nonzero value between February 2 and February 12, zero before and after this date unless it is close to another special day, and its maximum value of 1 on February 8. The dataset also includes operating system, browser, region, traffic type, visitor type as returning or new visitor, a Boolean value indicating whether the date of the visit is weekend, and month of the year.



In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12330 entries, 0 to 12329
Data columns (total 18 columns):
Administrative             12330 non-null int64
Administrative_Duration    12330 non-null float64
Informational              12330 non-null int64
Informational_Duration     12330 non-null float64
ProductRelated             12330 non-null int64
ProductRelated_Duration    12330 non-null float64
BounceRates                12330 non-null float64
ExitRates                  12330 non-null float64
PageValues                 12330 non-null float64
SpecialDay                 12330 non-null float64
Month                      12330 non-null object
OperatingSystems           12330 non-null int64
Browser                    12330 non-null int64
Region                     12330 non-null int64
TrafficType                12330 non-null int64
VisitorType                12330 non-null object
Weekend                    12330 non-null bool
Revenue                    12330 non-null bool
dtypes: bool(

## 4.3 Screen for Categorical variables:


In [6]:
df.head()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue
0,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,Feb,1,1,1,1,Returning_Visitor,False,False
1,0,0.0,0,0.0,2,64.000000,0.00,0.10,0.0,0.0,Feb,2,2,1,2,Returning_Visitor,False,False
2,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,Feb,4,1,9,3,Returning_Visitor,False,False
3,0,0.0,0,0.0,2,2.666667,0.05,0.14,0.0,0.0,Feb,3,2,2,4,Returning_Visitor,False,False
4,0,0.0,0,0.0,10,627.500000,0.02,0.05,0.0,0.0,Feb,3,3,1,4,Returning_Visitor,True,False


In [7]:
# Here we are checking the ratio of unique values to the total number count for each column
df.nunique()/df.count()
# proportions of nuniques to total counts < 0.05 suggest categorical variables

Administrative             0.002190
Administrative_Duration    0.270479
Informational              0.001379
Informational_Duration     0.102028
ProductRelated             0.025223
ProductRelated_Duration    0.774615
BounceRates                0.151825
ExitRates                  0.387429
PageValues                 0.219303
SpecialDay                 0.000487
Month                      0.000811
OperatingSystems           0.000649
Browser                    0.001054
Region                     0.000730
TrafficType                0.001622
VisitorType                0.000243
Weekend                    0.000162
Revenue                    0.000162
dtype: float64

In [8]:
#Loop to visually inspect value counts for all variables
# for col in df.columns:
#     print(f'This is {col} value counts: \n{df[col].value_counts()}.\n')

In [9]:
# Subset categorical and continuous features from dataframe for visualisations:
df_cont = df[['Administrative_Duration', 'Informational_Duration', 'ProductRelated_Duration', 'ExitRates', 'PageValues', 'BounceRates']]
df_cat = df.drop(df_cont.columns, axis=1)

# 5. Baseline Model

## 5.1 Decision Tree

In [10]:
# Feature and Target subsetting:
X = df.drop(columns=['Revenue'], axis =1)
y = df['Revenue']

In [11]:
#train_test_split and create a global seed
seed=42
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state = seed)

In [12]:
# One-hot encode the categorical variables in the training data and show the resulting DataFrame with proper column names
ohe = OneHotEncoder()
#subset continuous and categorical variables:
train_cont = X_train[['Administrative_Duration', 'Informational_Duration', 'ProductRelated_Duration', 'ExitRates', 'PageValues', 'BounceRates']]
train_cat = X_train.drop(df_cont.columns, axis=1)


#Fit transform the variables and place them in a dataframe
encoded_vars = ohe.fit_transform(train_cat).toarray()
ohe_df = pd.DataFrame(encoded_vars, columns=ohe.get_feature_names(train_cat.columns))
ohe_df.shape


(9864, 401)

In [13]:
# reset index and make a copy of continuous dataframe.
X_train_ohe = train_cont.copy()
X_train_ohe.reset_index(drop=True, inplace=True)

# Concat into continuous and encoded categoricals into one training dataset:
X_train_ohe = pd.concat([X_train_ohe, ohe_df], axis=1)
X_train_ohe.shape

(9864, 407)

## 5.2 Decision Tree fit

In [14]:
# Decision tree classifier fit 
clf = DecisionTreeClassifier(random_state = seed, criterion='entropy')

clf.fit(X_train_ohe, y_train)
#KFold cross validator instantiated:
cv = KFold(n_splits=5, random_state=seed)

# Use Crossvalidation to obtain Performance metrics: F1 Score and Accuracy
F1_score = round(np.mean(cross_val_score(clf, X_train_ohe, y_train, cv=cv, scoring='f1')), 4)
Acc_score = round(np.mean(cross_val_score(clf, X_train_ohe, y_train, cv=cv, scoring='accuracy')), 4)
roc_AUC_score = round(np.mean(cross_val_score(clf, X_train_ohe, y_train, cv=cv, scoring='roc_auc')), 4)
#Print F1 and Accuracy and ROC_AUC score for crossvalidation baseline decision tree
F1_score, Acc_score, roc_AUC_score

(0.5674, 0.8692, 0.7445)

In [15]:
## 5.4 Baseline Model - Plot the Decision Tree

# Create DOT Data
# dot_data = export_graphviz(clf, out_file=None,
#                            feature_names=X_train_ohe.columns,
#                            class_names=np.unique(y).astype('str'),
#                            filled=True, rounded=True, special_characters=True)

# Draw Graph 
# graph = graph_from_dot_data(dot_data) 

# Show graph
# Image(graph.create_png())


# 6.0 Exploring Potential Improvements on Baseline Model



## 6.1 Decision tree with Gini impurity

In [16]:
# Gini impurity Decision tree classifier fit 
clf1 = DecisionTreeClassifier(random_state = seed, criterion='gini')
clf1.fit(X_train_ohe, y_train)

# Use Crossvalidation to obtain Performance metrics: F1 Score and Accuracy
F1_score = round(np.mean(cross_val_score(clf1, X_train_ohe, y_train, cv=cv, scoring='f1')), 4)
Acc_score = round(np.mean(cross_val_score(clf1, X_train_ohe, y_train, cv=cv, scoring='accuracy')), 4)
roc_AUC_score = round(np.mean(cross_val_score(clf1, X_train_ohe, y_train, cv=cv, scoring='roc_auc')), 4)

#Print F1 and Accuracy and ROC_AUC score for crossvalidation baseline decision tree
F1_score, Acc_score, roc_AUC_score

(0.5622, 0.8688, 0.7403)

## 6.2 Grid Search CV for Decision tree with Entropy impurity

In [17]:
# Entropy impurity performed better. Let's implement hyperparameter tuning with combinatoric grid searching.

#initial Param grid:

dt_param_grid = {
    'criterion': ['entropy'],
    'max_depth': [None, 2, 3, 4, 5, 6],
    'min_samples_split': [10, 500, 1000]
}


#insantiate the GridSearchCV
dt_grid_search = GridSearchCV(clf, dt_param_grid, cv=3, return_train_score=True)

# Fit to the data
dt_grid_search.fit(X_train_ohe, y_train)

GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=DecisionTreeClassifier(class_weight=None,
                                              criterion='entropy',
                                              max_depth=None, max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort=False, random_state=42,
                                              splitter='best'),
             iid='warn', n_jobs=None,
             param_grid={'criterion': ['entropy'],
                         'max_depth': [None, 2, 3, 4, 5, 6],
                       

In [18]:
# Mean training score
dt_gs_training_score = np.mean(dt_grid_search.cv_results_['mean_train_score'])


print(f"Mean Training Score: {dt_gs_training_score :.2%}")

# Print best parameter combination found during grid search:
print("Best Parameter Combination Found During Grid Search:")
dt_grid_search.best_params_

Mean Training Score: 90.32%
Best Parameter Combination Found During Grid Search:


{'criterion': 'entropy', 'max_depth': 3, 'min_samples_split': 10}

In [19]:
# Primary Param grid prediction scores: (Accuracy, Precision, Recall and F1)
predictions = dt_grid_search.best_estimator_.predict(X_train_ohe)
accuracy_score1 = accuracy_score(y_train, predictions)
recall_score1 = recall_score(y_train, predictions)
precision_score1 = precision_score(y_train, predictions)
F1_score1 = f1_score(y_train, predictions)
print(f'Recall Score:{recall_score1}, Precision Score:{precision_score1}, Accuracy Score{accuracy_score1}, F1 Score {F1_score1}')

Recall Score:0.6239144956579826, Precision Score:0.6949404761904762, Accuracy Score0.9013584752635847, F1 Score 0.6575149595212954


In [20]:
# Secondary Param_Grid:

dt_param_grid1 = {
    'criterion': ['entropy'],
    'max_depth': [3],
    'min_samples_split': [2,3,5,7,9]
}
dt_grid_search1 = GridSearchCV(clf, dt_param_grid1, cv=3, return_train_score=True)

# Fit to the data
dt_grid_search1.fit(X_train_ohe, y_train)

GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=DecisionTreeClassifier(class_weight=None,
                                              criterion='entropy',
                                              max_depth=None, max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort=False, random_state=42,
                                              splitter='best'),
             iid='warn', n_jobs=None,
             param_grid={'criterion': ['entropy'], 'max_depth': [3],
                         'min_samples_split': [2, 3, 5, 7, 9]},
  

In [21]:
# Secondary Param Grid Mean train score 
dt_gs_training_score1 = np.mean(dt_grid_search1.cv_results_['mean_train_score'])

print(f"Mean Training Score: {dt_gs_training_score1 :.2%}")

# Secondary Param grid best combinations:
print("Best Parameter Combination Found During Grid Search:")
dt_grid_search1.best_params_

Mean Training Score: 90.04%
Best Parameter Combination Found During Grid Search:


{'criterion': 'entropy', 'max_depth': 3, 'min_samples_split': 2}

In [22]:
#Predictions for secondary parameter grid and its related Recall, precision accuracy and F1 scores:
predictions = dt_grid_search1.best_estimator_.predict(X_train_ohe)
acc_score = accuracy_score(y_train, predictions)
rec_score = recall_score(y_train, predictions)
prec_score = precision_score(y_train, predictions)
F1_score = f1_score(y_train, predictions)
print(f'Recall Score:{rec_score}, Precision Score:{prec_score}, Accuracy Score{acc_score}, F1 Score {F1_score}')

Recall Score:0.6239144956579826, Precision Score:0.6949404761904762, Accuracy Score0.9013584752635847, F1 Score 0.6575149595212954


In [ ]:
# #Roc plot:
# scores(model, X_train_ohe,y_train)
# roc_plot(model,X_train_ohe,y_train)

# 6.3 Ensemble Methods - Random Forests

In [24]:
# Instantiate Random Forest and cross validate fit with the training data:
rf_clf = RandomForestClassifier()
mean_rf_cv_score = np.mean(cross_val_score(rf_clf, X_train_ohe, y_train, scorincv=3))

rf_clf.fit(X_train_ohe, y_train)
#print resulting mean score.
print(f"Mean Cross Validation Score for Random Forest Classifier: {mean_rf_cv_score : .2%}")

#Predictions and scores for out-of-box Random forest:
predictions = dt_grid_search1.best_estimator_.predict(X_train_ohe)
acc_score = accuracy_score(y_train, predictions)
rec_score = recall_score(y_train, predictions)
prec_score = precision_score(y_train, predictions)
F1_score = f1_score(y_train, predictions)
print(f'Recall Score:{rec_score}, Precision Score:{prec_score}, Accuracy Score{acc_score}, F1 Score {F1_score}')

/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Mean Cross Validation Score for Random Forest Classifier:  88.14%
Recall Score:0.6239144956579826, Precision Score:0.6949404761904762, Accuracy Score0.9013584752635847, F1 Score 0.6575149595212954


In [44]:
#Random Forest Param Grid 1:

# Create the Random_Grid
random_grid = {'bootstrap': [True, False],
             'max_depth': [10, 20, 30, 40, None],
             'max_features': ['auto', 'sqrt'],
             'min_samples_split': [2, 5, 10],
             'n_estimators': [100, 200]
              }

In [45]:
# Use the random grid to search for best hyperparameters

# Random search of parameters, using 3 fold cross validation, 
# Will search across 1000 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf_clf, param_distributions = random_grid, n_iter = 1000, cv = 3, verbose=2, scoring='roc_auc', random_state=seed, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train_ohe, y_train)

Fitting 3 folds for each of 900 candidates, totalling 2700 fits


/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:266: UserWarning: The total space of parameters 900 is smaller than n_iter=1000. Running 900 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  7.5min
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed: 27.2min
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed: 45.9min
[Parallel(n_jobs=-1)]: Done 1005 tasks      | elapsed: 72.0min
[Parallel(n_jobs=-1)]: Done 1450 tasks      | elapsed: 114.9min
[Parallel(n_jobs=-1)]: Done 1977 tasks      | elapsed: 207.7min
[Parallel(n_jobs=-1)]: Done 2584 tasks      | elapsed: 815.6min
[Parallel(n_jobs=-1)]: Done 2700 out of 2700 | elapsed: 827.9min finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    n_estimators='warn',
                                                    n_jobs=None

In [25]:
#Predictions for secondary parameter grid and its related Recall, precision accuracy and F1 scores:
predictions = rf_random.best_estimator_.predict(X_train_ohe)
acc_score = accuracy_score(y_train, predictions)
rec_score = recall_score(y_train, predictions)
prec_score = precision_score(y_train, predictions)
F1_score = f1_score(y_train, predictions)
print(f'Recall Score:{rec_score}, Precision Score:{prec_score}, Accuracy Score{acc_score}, F1 Score {F1_score}')

NameError: name 'rf_random' is not defined

In [47]:
rf_random.best_params_

{'n_estimators': 400,
 'min_samples_split': 5,
 'min_samples_leaf': 1,
 'max_features': 'auto',
 'max_depth': 40,
 'bootstrap': False}

In [26]:
# GridSearchCV Param_grid Based on Random Search results:


rf_param_grid = {
    'bootstrap': [False],
    'max_depth': [20, 30, 40, 50],
    'max_features': [2, 3, 4],
    'min_samples_leaf': [3, 4, 5],
    'n_estimators': [200, 300, 400]
}

In [29]:
#Instantiate and fit Grid Search 1
grid_search = GridSearchCV(estimator=rf_clf, param_grid=rf_param_grid, n_jobs= -1, scoring='roc_auc', verbose=2, cv=3)
grid_search.fit(X_train_ohe,y_train)

Fitting 3 folds for each of 108 candidates, totalling 324 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   20.6s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 324 out of 324 | elapsed:  3.3min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=10, n_jobs=None,
                                              oob_score=False,
                                              random_state=None, verbose=0,
                                              warm_start=False),
             iid='wa

In [30]:
grid_search.best_params_

{'bootstrap': False,
 'max_depth': 40,
 'max_features': 4,
 'min_samples_leaf': 3,
 'n_estimators': 300}

In [31]:
grid_search.best_score_

0.898364914363207

In [38]:
# secondary GridSearchCV:


rf_param_grid = {
    'criterion': ['entropy'],
    'bootstrap': [False],
    'max_depth': [40, 45, 50],
    'max_features': [4, 5],
    'min_samples_leaf': [1, 2, 3],
    'n_estimators': [300, 350]
}

In [40]:
#Instantiate and fit Grid Search 2
grid_search = GridSearchCV(estimator=rf_clf, param_grid=rf_param_grid, n_jobs= -1, scoring='roc_auc', verbose=2, cv=3)
grid_search.fit(X_train_ohe, y_train)

Fitting 3 folds for each of 36 candidates, totalling 108 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 108 out of 108 | elapsed:  2.8min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=10, n_jobs=None,
                                              oob_score=False,
                                              random_state=None, verbose=0,
                                              warm_start=False),
             iid='wa

In [44]:
grid_search.best_params_


{'bootstrap': False,
 'criterion': 'entropy',
 'max_depth': 40,
 'max_features': 5,
 'min_samples_leaf': 1,
 'n_estimators': 350}

In [43]:
grid_search.best_score_

0.9138123264576243

In [45]:
# Tertiary GridSearchCV:


rf_param_grid = {
    'criterion': ['entropy'],
    'bootstrap': [False],
    'max_depth': [40],
    'max_features': [5, 6, 7],
    'min_samples_leaf': [1],
    'n_estimators': [350, 400, 450]
}

In [46]:
#Instantiate and fit Grid Search 3
grid_search = GridSearchCV(estimator=rf_clf, param_grid=rf_param_grid, n_jobs= -1, scoring='roc_auc', verbose=2, cv=3)
grid_search.fit(X_train_ohe, y_train)

Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed:  1.6min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=10, n_jobs=None,
                                              oob_score=False,
                                              random_state=None, verbose=0,
                                              warm_start=False),
             iid='wa

In [47]:
grid_search.best_params_

{'bootstrap': False,
 'criterion': 'entropy',
 'max_depth': 40,
 'max_features': 7,
 'min_samples_leaf': 1,
 'n_estimators': 350}

In [48]:
grid_search.best_score_

0.9155076017937632

In [49]:
# Tertiary GridSearchCv
rf_param_grid = {
    'criterion': ['entropy'],
    'bootstrap': [False],
    'max_depth': [40],
    'max_features': [7, 8, 9, 10, 11],
    'min_samples_leaf': [1],
    'n_estimators': [350]
}
#Instantiate and fit Grid Search 2
grid_search = GridSearchCV(estimator=rf_clf, param_grid=rf_param_grid, n_jobs= -1, scoring='roc_auc', verbose=2, cv=3)
grid_search.fit(X_train_ohe, y_train)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:  1.1min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=10, n_jobs=None,
                                              oob_score=False,
                                              random_state=None, verbose=0,
                                              warm_start=False),
             iid='wa

In [50]:
grid_search.best_params_

{'bootstrap': False,
 'criterion': 'entropy',
 'max_depth': 40,
 'max_features': 11,
 'min_samples_leaf': 1,
 'n_estimators': 350}

In [51]:
grid_search.best_score_

0.9187968383282642

In [57]:
# quaternary GridSearchCv
rf_param_grid = {
    'criterion': ['entropy'],
    'bootstrap': [False],
    'max_depth': [40],
    'max_features': [10, 50, 200, 400],
    'min_samples_leaf': [1],
    'n_estimators': [350]
}
#Instantiate and fit Grid Search 2
grid_search = GridSearchCV(estimator=rf_clf, param_grid=rf_param_grid, n_jobs= -1, scoring='roc_auc', verbose=2, cv=3)
grid_search.fit(X_train_ohe, y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:  2.8min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:  2.8min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=10, n_jobs=None,
                                              oob_score=False,
                                              random_state=None, verbose=0,
                                              warm_start=False),
             iid='wa

In [58]:
grid_search.best_params_

{'bootstrap': False,
 'criterion': 'entropy',
 'max_depth': 40,
 'max_features': 50,
 'min_samples_leaf': 1,
 'n_estimators': 350}

In [59]:
grid_search.best_score_

0.9271251957722065

In [60]:
# Fifth GridSearchCv
rf_param_grid = {
    'criterion': ['entropy'],
    'bootstrap': [False],
    'max_depth': [40],
    'max_features': [30, 50,70, 90],
    'min_samples_leaf': [1],
    'n_estimators': [350]
}
#Instantiate and fit Grid Search 2
grid_search = GridSearchCV(estimator=rf_clf, param_grid=rf_param_grid, n_jobs= -1, scoring='roc_auc', verbose=2, cv=3)
grid_search.fit(X_train_ohe, y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:  1.6min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:  1.6min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=10, n_jobs=None,
                                              oob_score=False,
                                              random_state=None, verbose=0,
                                              warm_start=False),
             iid='wa

In [61]:
grid_search.best_params_

{'bootstrap': False,
 'criterion': 'entropy',
 'max_depth': 40,
 'max_features': 70,
 'min_samples_leaf': 1,
 'n_estimators': 350}

In [63]:
grid_search.best_score_

0.9279106797316398

In [64]:
# sixth GridSearchCv
rf_param_grid = {
    'criterion': ['entropy'],
    'bootstrap': [False],
    'max_depth': [40],
    'max_features': [65, 70,75],
    'min_samples_leaf': [1],
    'n_estimators': [350]
}
#Instantiate and fit Grid Search 2
grid_search = GridSearchCV(estimator=rf_clf, param_grid=rf_param_grid, n_jobs= -1, scoring='roc_auc', verbose=2, cv=3)
grid_search.fit(X_train_ohe, y_train)

Fitting 3 folds for each of 3 candidates, totalling 9 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   7 out of   9 | elapsed:  1.1min remaining:   18.6s
[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed:  1.3min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=10, n_jobs=None,
                                              oob_score=False,
                                              random_state=None, verbose=0,
                                              warm_start=False),
             iid='wa

In [65]:
grid_search.best_params_

{'bootstrap': False,
 'criterion': 'entropy',
 'max_depth': 40,
 'max_features': 65,
 'min_samples_leaf': 1,
 'n_estimators': 350}

In [66]:
grid_search.best_score_

0.9274560834349468

# 7.0 Threshold Selection

In [80]:
# Predict probabilities from optimal model:
from sklearn.model_selection import cross_val_predict
optimal_model = grid_search.best_estimator_
probas = optimal_model.predict_proba(X_train_ohe)

y_train_rf_predict = cross_val_predict(optimal_model, X_train_ohe, y_train, cv=3)



In [81]:
cm_rf = confusion_matrix(y_train, y_train_rf_predict)
cm_rf


array([[8066,  301],
       [ 653,  844]])

In [90]:
y_prob = optimal_model.predict_proba(X_train_ohe)[:, 1]

In [91]:
y_prob


array([0., 0., 0., ..., 0., 0., 0.])

In [93]:
prec_all, rec_all, thresh_all = precision_recall_curve(y_train, y_prob)

In [98]:
thresh_all

array([0.92210699, 0.93863438, 0.94329093, 0.94536578, 0.94973476,
       0.95143379, 0.95205264, 0.95446465, 0.95481273, 0.95679811,
       0.95703392, 0.95874261, 0.96106551, 0.96190695, 0.96429791,
       0.96464603, 0.96525592, 0.96741941, 0.96774547, 0.96839081,
       0.96887903, 0.96973557, 0.97135635, 0.97228017, 0.97261951,
       0.97372301, 0.97462812, 0.97524843, 0.97529611, 0.97567869,
       0.97717489, 0.97769117, 0.97783026, 0.97827895, 0.98035828,
       0.98048471, 0.9812662 , 0.98141428, 0.98250854, 0.98272179,
       0.98280017, 0.98284611, 0.98299066, 0.98342241, 0.9839184 ,
       0.98408189, 0.98468609, 0.98509078, 0.98532199, 0.98543768,
       0.98625749, 0.98639401, 0.98652731, 0.98654814, 0.98671251,
       0.98714951, 0.98758339, 0.98782624, 0.98794453, 0.98805442,
       0.98809693, 0.98815159, 0.98834328, 0.98835309, 0.98845921,
       0.98886429, 0.98889139, 0.98897545, 0.98901058, 0.98942595,
       0.98965356, 0.98997562, 0.99028345, 0.99028938, 0.99048